cs model v.00 (big balz)

In [15]:
import numpy as np
import pandas as pd
import time
start_time = time.time()

In [2]:
# read the dataset + drop the Date column
# df = pd.read_csv(r'tsla_stock_data_with_features.csv').drop(columns=['Date'], axis=1)
# df = pd.read_csv(r'tsla_stock_data_with_features_1y.csv').drop(columns=['Date'], axis=1)
df = pd.read_csv(r'tsla_stock_data_with_features_2y.csv').drop(columns=['Date'], axis=1)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3448 entries, 0 to 3447
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Close            3448 non-null   float64
 1   High             3448 non-null   float64
 2   Low              3448 non-null   float64
 3   Open             3448 non-null   float64
 4   Volume           3448 non-null   int64  
 5   Daily_Return     3448 non-null   float64
 6   Price_Momentum   3448 non-null   float64
 7   Volatility       3448 non-null   float64
 8   Pct_Change       3448 non-null   float64
 9   SMA_5            3448 non-null   float64
 10  SMA_10           3448 non-null   float64
 11  EMA_5            3448 non-null   float64
 12  EMA_10           3448 non-null   float64
 13  Volume_Change    3448 non-null   float64
 14  RSI_14           3448 non-null   float64
 15  Hour             3448 non-null   int64  
 16  DayOfWeek        3448 non-null   int64  
 17  MACD          

In [4]:
#Create a target variable: Next period's Close price
#    (This means we're predicting what the close price will be in the next time interval)
df['Target_Close'] = df['Close'].shift(-1)

# Drop the last row which will have NaN for Target_Close
df = df.dropna()

In [5]:
X = df.drop(['Target_Close'], axis=1)
y = df['Target_Close']

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

X.corr()

,Close,High,Low,Open,Volume,Daily_Return,Price_Momentum,Volatility,Pct_Change,SMA_5,...,Stochastic_%D,Lag_Close_1,Lag_Close_2,Lag_Close_3,Lag_Close_4,Lag_Close_5,Lag_Close_6,Lag_Close_7,Lag_Close_8,Lag_Close_9
Close,1.000000,0.999594,0.999662,0.999267,-0.066021,0.007592,0.032056,0.123984,0.029732,0.998602,...,0.090948,0.998827,0.997667,0.996533,0.995317,0.994016,0.992739,0.991453,0.990104,0.988749
High,0.999594,1.000000,0.999394,0.999662,-0.054190,-0.011500,0.016291,0.142081,0.015757,0.998854,...,0.087687,0.999184,0.998033,0.996985,0.995807,0.994563,0.993289,0.991995,0.990607,0.989258
Low,0.999662,0.999394,1.000000,0.999570,-0.078573,-0.009144,0.017950,0.109158,0.016244,0.998791,...,0.089135,0.999172,0.997975,0.996855,0.995626,0.994361,0.993043,0.991731,0.990440,0.989098
Open,0.999267,0.999662,0.999570,1.000000,-0.066537,-0.028920,0.002567,0.127475,0.002642,0.999027,...,0.086605,0.999522,0.998351,0.997257,0.996072,0.994858,0.993542,0.992214,0.990803,0.989453
Volume,-0.066021,-0.054190,-0.078573,-0.066537,1.000000,0.027953,0.071897,0.748203,0.084511,-0.070038,...,0.080118,-0.069529,-0.071123,-0.071249,-0.071946,-0.072561,-0.072840,-0.073776,-0.077285,-0.079093
Daily_Return,0.007592,-0.011500,-0.009144,-0.028920,0.027953,1.000000,0.739833,-0.058071,0.743532,-0.021278,...,0.116974,-0.028252,-0.028078,-0.028420,-0.029158,-0.031187,-0.030312,-0.029201,-0.028016,-0.028156
Price_Momentum,0.032056,0.016291,0.017950,0.002567,0.071897,0.739833,1.000000,-0.042203,0.958077,-0.006640,...,0.147059,-0.016383,-0.016091,-0.015563,-0.017225,-0.018938,-0.018399,-0.018579,-0.019877,-0.019985
Volatility,0.123984,0.142081,0.109158,0.127475,0.748203,-0.058071,-0.042203,1.000000,-0.017413,0.127043,...,-0.025484,0.126076,0.127030,0.128313,0.129225,0.129865,0.130469,0.130824,0.128554,0.128307
Pct_Change,0.029732,0.015757,0.016244,0.002642,0.084511,0.743532,0.958077,-0.017413,1.000000,-0.007258,...,0.150850,-0.016677,-0.016492,-0.015765,-0.017092,-0.018547,-0.018079,-0.017981,-0.019781,-0.019828
SMA_5,0.998602,0.998854,0.998791,0.999027,-0.070038,-0.021278,-0.006640,0.127043,-0.007258,1.000000,...,0.068064,0.999303,0.999529,0.999302,0.998598,0.997397,0.996174,0.994927,0.993637,0.992320


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Gonna test different models
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
def compare_regularized_models(X_train_scaled, y_train, X_test_scaled, y_test):
    # Define parameter grids for each model
    param_grid_ridge = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}
    param_grid_lasso = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100],  'max_iter': [1000, 10000, 50000, 100000, 200000]}
    param_grid_elastic = {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'l1_ratio': [0.1, 0.5, 0.9],
        'max_iter': [1000, 10000, 50000, 100000, 200000]
    }

    # Initialize models
    ridge = Ridge()
    lasso = Lasso()
    elastic = ElasticNet()

    # Setup GridSearchCV for each
    grid_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_elastic = GridSearchCV(elastic, param_grid_elastic, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

    # Fit models on the training data
    grid_ridge.fit(X_train, y_train)
    grid_lasso.fit(X_train, y_train)
    grid_elastic.fit(X_train, y_train)

    # Get best estimators
    best_ridge = grid_ridge.best_estimator_
    best_lasso = grid_lasso.best_estimator_
    best_elastic = grid_elastic.best_estimator_

    # Predict on test set
    pred_ridge = best_ridge.predict(X_test)
    pred_lasso = best_lasso.predict(X_test)
    pred_elastic = best_elastic.predict(X_test)

    # Evaluate performance
    results = {
        'Model': [],
        'Best Params': [],
        'Test MSE': [],
        'Test RMSE': [],
        'Test R2': []
    }

    for model_name, best_model, pred in [
        ("Ridge", best_ridge, pred_ridge),
        ("Lasso", best_lasso, pred_lasso),
        ("ElasticNet", best_elastic, pred_elastic)
    ]:
        mse = mean_squared_error(y_test, pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, pred)
        results['Model'].append(model_name)
        results['Best Params'].append(best_model.get_params())
        results['Test MSE'].append(mse)
        results['Test RMSE'].append(rmse)
        results['Test R2'].append(r2)

    results_df = pd.DataFrame(results)
    print("Comparison of Regularized Models:")
    print(results_df)
    return results_df

In [11]:
results_df = compare_regularized_models(X_train_scaled, y_train, X_test_scaled, y_test)

C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.1176e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Comparison of Regularized Models:
        Model                                        Best Params  Test MSE  \
0       Ridge  {'alpha': 100, 'copy_X': True, 'fit_intercept'...  7.943182   
1       Lasso  {'alpha': 1, 'copy_X': True, 'fit_intercept': ...  7.542194   
2  ElasticNet  {'alpha': 1, 'copy_X': True, 'fit_intercept': ...  7.549999   

    Test R2  
0  0.998341  
1  0.998425  
2  0.998423  


In [12]:
results_df.to_csv(r'2_year_model_comparison_results', index=False)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")

Execution time: 220.2051 seconds


In [13]:
pd.read_csv(r'2_year_model_comparison_results')

,Model,Best Params,Test MSE,Test R2
0,Ridge,"{'alpha': 100, 'copy_X': True, 'fit_intercept'...",7.943182,0.998341
1,Lasso,"{'alpha': 1, 'copy_X': True, 'fit_intercept': ...",7.542194,0.998425
2,ElasticNet,"{'alpha': 1, 'copy_X': True, 'fit_intercept': ...",7.549999,0.998423


In [17]:
# looking into lasso regression solo
param_grid_lasso = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100], 'max_iter': [1000, 10000, 50000, 100000, 200000]}

lasso = Lasso()

grid_lasso = GridSearchCV(lasso, param_grid_lasso, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

grid_lasso.fit(X_train, y_train)

best_lasso = grid_lasso.best_estimator_

pred_lasso = best_lasso.predict(X_test)

results = {
        'Model': [],
        'Best Params': [],
        'Test MSE': [],
        'Test RMSE': [],
        'Test R2': [],
        'Best estimator': [best_lasso]
    }

for model_name, best_model, pred in [
        ("Lasso", best_lasso, pred_lasso),
    ]:
        mse = mean_squared_error(y_test, pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, pred)
        results['Model'].append(model_name)
        results['Best Params'].append(best_model.get_params())
        results['Test MSE'].append(mse)
        results['Test RMSE'].append(rmse)
        results['Test R2'].append(r2)

print(pd.DataFrame(results))

   Model                                        Best Params  Test MSE  \
0  Lasso  {'alpha': 1, 'copy_X': True, 'fit_intercept': ...  7.540228   

   Test RMSE   Test R2  Best estimator  
0   2.745948  0.998425  Lasso(alpha=1)  


C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.040e+04, tolerance: 1.426e+03
  model = cd_fast.enet_coordinate_descent(


In [18]:
results = pd.DataFrame(results)

In [21]:
results['Best Params'][0]

{'alpha': 1,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [23]:
# train lasso on all data
lasso = Lasso(alpha=1, max_iter=1000)

lasso.fit(scaler.transform(X), y)

Lasso(alpha=1)

In [29]:
df['Target_Close'].iloc[0]

208.06869506835932

In [31]:
X_new = df.drop(columns=['Target_Close'], axis=1).iloc[0].values.reshape(1, -1)

predictions = lasso.predict(scaler.transform(X_new))
predictions

C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([209.61599133])

In [32]:
df['Open'].iloc[0]

207.1699981689453

In [39]:
import random
random_integers = [random.randint(0,3447) for _ in range(10)]

table = [['Open', 'Target_Close', 'Predicted_Close']]

for i in random_integers:
    X_new = df.drop(columns=['Target_Close'], axis=1).iloc[i].values.reshape(1, -1)
    predictions = lasso.predict(scaler.transform(X_new))
    
    table.append([df['Open'].iloc[i], df['Target_Close'].iloc[i], predictions[0]])

C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\usual\anaconda3\envs\ML_course\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\usual\anaconda3\

In [43]:
pd.DataFrame(table[1:], columns=table[0], index=range(len(table)-1))

,Open,Target_Close,Predicted_Close
0,449.578094,455.512909,449.415514
1,254.250000,254.029999,254.435548
2,235.750000,242.574997,240.954598
3,177.229996,175.919998,177.499676
4,199.720001,205.320007,202.629643
5,248.475006,249.417999,249.680308
6,147.199997,141.240097,148.474808
7,164.529999,158.619995,162.564126
8,257.730011,256.855011,257.251975
9,236.759995,236.970001,237.008647
